In [ ]:
#| default_exp core

# fastkaggle.core

> API details for fastkaggle.

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import os,json,subprocess
from fastcore.utils import *


In [ ]:
#|export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
#|export
def import_kaggle():
    "Import kaggle API, using Kaggle secrets `kaggle_username` and `kaggle_key` if needed"
    if iskaggle:
        from kaggle_secrets import UserSecretsClient
        sec = UserSecretsClient()
        os.environ['KAGGLE_USERNAME'] = sec.get_secret("kaggle_username")
        if not os.environ['KAGGLE_USERNAME']: raise Exception("Please insert your Kaggle username and key into Kaggle secrets")
        os.environ['KAGGLE_KEY'] = sec.get_secret("kaggle_key")
    from kaggle import api
    return api

In [ ]:
api = import_kaggle()
L(api.competitions_list())

(#20) [contradictory-my-dear-watson,gan-getting-started,store-sales-time-series-forecasting,tpu-getting-started,digit-recognizer,titanic,house-prices-advanced-regression-techniques,connectx,nlp-getting-started,spaceship-titanic...]

In [ ]:
#|export
def setup_comp(competition, install=''):
    "Get a path to data for `competition`, downloading it if needed"
    if iskaggle:
        if install:
            os.system(f'pip install -Uqq {install}')
        return Path('../input')/competition
    else:
        path = Path(competition)
        from kaggle import api
        if not path.exists():
            import zipfile
            api.competition_download_cli(str(competition))
            zipfile.ZipFile(f'{competition}.zip').extractall(str(competition))
        return path

In [ ]:
setup_comp('titanic')

100%|██████████| 34.1k/34.1k [00:00<00:00, 12.9MB/s]

Path('titanic')

If you pass a list of space separated modules to `install`, they'll be installed if running on Kaggle.

In [ ]:
#|export
def nb_meta(user, id, title, file, competition=None, private=True, gpu=False, internet=True):
    "Get the `dict` required for a kernel-metadata.json file"
    d = {
      "id": f"{user}/{id}",
      "title": title,
      "code_file": file,
      "language": "python",
      "kernel_type": "notebook",
      "is_private": private,
      "enable_gpu": gpu,
      "enable_internet": internet,
      "keywords": [],
      "dataset_sources": [],
      "kernel_sources": []
    }
    if competition: d["competition_sources"] = [f"competitions/{competition}"]
    return d

In [ ]:
nb_meta('jhoward', 'my-notebook', 'My notebook', 'my-notebook.ipynb', competition='paddy-disease-classification')

{'id': 'jhoward/my-notebook',
 'title': 'My notebook',
 'code_file': 'my-notebook.ipynb',
 'language': 'python',
 'kernel_type': 'notebook',
 'is_private': True,
 'enable_gpu': False,
 'enable_internet': True,
 'keywords': [],
 'dataset_sources': [],
 'kernel_sources': [],
 'competition_sources': ['competitions/paddy-disease-classification']}

In [ ]:
#|export
def push_notebook(user, id, title, file, path='.', competition=None, private=True, gpu=False, internet=True):
    "Push notebook `file` to Kaggle Notebooks"
    meta = nb_meta(user, id, title, file=file, competition=competition, private=private, gpu=gpu, internet=internet)
    path = Path(path)
    nm = 'kernel-metadata.json'
    path.mkdir(exist_ok=True, parents=True)
    with open(path/nm, 'w') as f: json.dump(meta, f, indent=2)
    from kaggle import api
    api.kernels_push_cli(str(path))

Note that Kaggle recommends that the `id` match the *slug* for the title -- i.e it should be the same as the title, but lowercase, no punctuation, and spaces replaced with dashes. E.g:

```python
push_notebook('jhoward', 'first-steps-road-to-the-top-part-1',
              title='First Steps: Road to the Top, Part 1',
              file='first-steps-road-to-the-top-part-1.ipynb',
              competition='paddy-disease-classification',
              private=False, gpu=True)
```

## Datasets

In [ ]:
#| export
def mk_dataset(dataset_path, # Local path to create dataset in
               title, # Name of the dataset
               force=False, # Should it overwrite or error if exists?
               upload=True # Should it upload and create on kaggle
              ):
    '''Creates minimal dataset metadata needed to push new dataset to kaggle'''
    dataset_path = Path(dataset_path)
    dataset_path.mkdir(exist_ok=force,parents=True)
    api.dataset_initialize(dataset_path)
    md = json.load(open(dataset_path/'dataset-metadata.json'))
    md['title'] = title
    md['id'] = md['id'].replace('INSERT_SLUG_HERE',title)
    json.dump(md,open(dataset_path/'dataset-metadata.json','w'))
    if upload: (dataset_path/'empty.txt').touch()
    api.dataset_create_new(str(dataset_path),public=True,dir_mode='zip')

In [ ]:
mk_dataset('./testds','mytestds',force=True)
md = json.load(open('./testds/dataset-metadata.json'))
assert md['title'] == 'mytestds'
assert md['id'].endswith('/mytestds')

Data package template written to: testds/dataset-metadata.json


In [ ]:
#| export
def get_dataset(dataset_path, # Local path to download dataset to
                dataset_slug, # Dataset slug (ie "zillow/zecon")
                unzip=True, # Should it unzip after downloading?
                force=False # Should it overwrite or error if dataset_path exists?
               ):
    '''Downloads an existing dataset and metadata from kaggle'''
    if not force: assert not Path(dataset_path).exists()
    api.dataset_metadata(dataset_slug,str(dataset_path))
    api.dataset_download_files(dataset_slug,str(dataset_path))
    if unzip:
        zipped_file = Path(dataset_path)/f"{dataset_slug.split('/')[-1]}.zip"
        import zipfile
        with zipfile.ZipFile(zipped_file, 'r') as zip_ref:
            zip_ref.extractall(Path(dataset_path))
        zipped_file.unlink()
    

In [ ]:
dl_path = Path('./mydataset')
get_dataset(dl_path,'isaacflath/testds',force=True)
ds_contents = ['abc.test','dataset-metadata.json','fastcore-1.5.11-py3-none-any.whl','packaging-21.3-py3-none-any.whl',
               'pip-22.2.1-py3-none-any.whl','pyparsing-3.0.9-py3-none-any.whl']
assert L(os.listdir(dl_path)).sorted() == ds_contents

In [ ]:
#| export
def get_pip_library(dataset_path, # Local path to download pip library to
                    pip_library, # name of library for pip to install
                    pip_cmd="pip" # pip base to use (ie "pip3" or "pip")
                   ):    
    '''Download the whl files for pip_library and store in dataset_path'''
    bashCommand = f"{pip_cmd} download {pip_library} -d {dataset_path}"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

In [ ]:
dl_path = Path('./mylib')
get_pip_library(dl_path,'fastkaggle')
assert 1==len([o for o in dl_path.ls() if str(o).startswith(f"{dl_path}/fastkaggle")])

In [ ]:
#| export
def push_dataset(dataset_path, # Local path where dataset is stored 
                 version_comment # Comment associated with this dataset update
                ):
    '''Push dataset update to kaggle.  Dataset path must contain dataset metadata file'''
    api.dataset_create_version(str(dataset_path),version_comment,dir_mode='zip')

In [ ]:
# dataset_path = Path('./testds452')
# mk_dataset(dataset_path,'mytestd232s',force=True)
# (dataset_path/'testfile.txt').touch()
# push_dataset(dataset_path,'testing')

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()